## Post Call Analytics (PCA) Using Amazon Bedrock
Amazon Bedrock을 사용한 통화 후 분석 사용 사례에 대한 이 교육에 오신 것을 환영합니다.

기업이 다양한 채널을 통해 고객과 계속 상호 작용함에 따라 이러한 상호 작용을 분석하여 고객 행동 및 선호도에 대한 인사이트를 얻는 것이 점점 더 중요해지고 있습니다.<BR>
통화 후 분석은 통화 종료 후 고객 상호 작용을 분석하는 방법 중 하나입니다.<BR>
대규모 언어 모델을 사용하면 보다 정확한 감정 분석을 가능하게 하고, 특정 고객의 요구와 선호도를 파악하며, 전반적인 고객 경험을 개선함으로써 통화 후 분석의 효과를 크게 높일 수 있습니다.<BR>

이 샘플 노트북에서는 통화 후 분석에 Bedrock을 사용할 때 얻을 수 있는 다양한 이점과 기업이 현대 시장에서 경쟁 우위를 확보할 수 있는 방법을 다음 주제를 통해 살펴봅니다.<BR>

- 베드락에서 LLM 모델 선택(타이탄 텍스트 및 앤트로닉 클로드)
- 하나의 모델로 여러 PCA 작업 처리
- 긴 통화 기록 처리

# 0. Auto Reload

In [ ]:
%load_ext autoreload
%autoreload 2

# 1. Import packages

In [ ]:
import os
import sys

module_path = "../../"
sys.path.append(os.path.abspath(module_path))

In [ ]:
import json
import time
import boto3
import librosa
import langchain
import numpy as np
import IPython.display as ipd
from time import strftime
from termcolor import colored
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock
import soundfile as sf
from utils import bedrock
from utils.s3 import s3_handler
from urllib.request import urlopen
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 2. Speech To Text (STT)

## 2.1. Run audio

In [ ]:
ipd.Audio("./records/voice-examples.wav", autoplay=False)

## 2.2. Upload data to s3

In [ ]:
s3 = s3_handler()

#### [중요] 아래 prefix 에 본인의 이름을 넣어 주세요. (예: gonsoomoon)
- bucket_name 이름은 유니크하기에, 다른 곳에서 사용을 하면 생성이 안됩니다.

In [ ]:
prefix = <your-id> #" gonsoomoon, mx-40"
bucket_name = f'bedrock-training-{prefix}'
data_dir = "./records"
data_path_s3 =f's3://{bucket_name}/records/voice-examples.wav'

In [ ]:
s3.create_bucket(bucket_name)
source_dir, target_bucket, target_dir = data_dir, bucket_name, "/records"
s3.upload_dir(source_dir, target_bucket, target_dir)


## 2.3. Transcribe

In [ ]:
transcribe_client = boto3.client('transcribe')

In [ ]:
create_date = strftime("%m%d-%H%M%s")
job_name = f'{prefix}-stt-job-{create_date}'
print (f's3 data path: {data_path_s3}')
print (f'job_name: {job_name}')

### 2.3.1. Run Transcribe Job

In [ ]:
transcribe_client.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': data_path_s3},
    MediaFormat='wav',
    Settings={
        'ShowSpeakerLabels': True,
        'MaxSpeakerLabels': 2,
    },
    LanguageCode='ko-KR'
)

### 2.3.2. Check Job Status

In [ ]:
while True:
    status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(10)
print(status)

In [ ]:
response = urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
data = json.loads(response.read())

### 2.3.3. Retrieve Text

In [ ]:
def spk_sperator(data):

    previos_spk = ""
    contents, contents_temp = [], []
    end_time = None

    for res in data["results"]["items"]:
        #print (res)
        speaker_label = res["speaker_label"]
        content = res["alternatives"][0]["content"]
        start_time = res.get("start_time", None)

        if previos_spk != speaker_label:

            contents_temp.append(f'<종료시간:{end_time}>')
            contents.append(" ".join(contents_temp))
            contents_temp = []

            contents_temp.append(f'{speaker_label}:<시작시간:{start_time}>')
            contents_temp.append(content)
        else:
            contents_temp.append(content)
            if content not in ["?", ",", "."]: end_time = res.get("end_time", None)

        previos_spk = speaker_label

    contents_temp.append(f'<종료시간:{end_time}>')
    contents.append(" ".join(contents_temp))

    return "\n".join(contents[1:])

if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    response = urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = json.loads(response.read())
    text = data['results']['transcripts'][0]['transcript']
    text = spk_sperator(data)
    print(text)

# 3. Post Call Analytics

## 3.1. Choice of models in Bedrock
Choose FMs from Amazon, AI21 Labs and Anthropic to find the right FM for your use case.

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

In [ ]:
# - create the Anthropic Model
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:", "\n\n인간", "\n\n상담원", "\n\n"],
        "temperature":0,
        "top_p":0.999,
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm

In [ ]:
max_tokens = {
    "Claude" : 100000,
    "TitanText": 4096,
    "Claude-instant": 9000,
    "Claude-V2" : 100000,
}

max_tokens = {"Claude" : 120, "TitanText": 130, "Claude-instant": 120, "Claude-V2" : 120}


## 3.2. Prompt Template
이 노트북에서는 네 가지 분석(**요약, 감성, 의도, 해결**)을 수행하게 되며, 각 분석에 대한 템플릿이 필요합니다.

In [ ]:
summary_template_ko = """\n\nHuman:
아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "{transcript}"

요약:

\n\nAssistant:"""

question_template_ko = """\n\nHuman:

아래의 통화 기록을 바탕으로 질문에 답하세요.
"<시작시간>" 이어지는 문장의 시작시간을 나타내고 "<종료시간>"은 앞 문장의 종료시간을 나타냅니다.

통화: "{transcript}"

질문: "{question}"

응답:

\n\nAssistant:"""

question_time_template_ko = """\n\nHuman:

아래의 통화 기록을 바탕으로 질문에 답하세요.
"<시작시간>" 이어지는 문장의 시작시간을 나타내고 "<종료시간>"은 앞 문장의 종료시간을 나타냅니다.

통화: "{transcript}"

질문: "{question} 답변과 함께 답변을 위해 참조한 대화의 시작 및 종료시간을 아래 형태로 알려주세요. \n답변:\n시작시간:\n종료시간:"

응답:

\n\nAssistant:"""


## 3.3. Generate Analysis

In [ ]:
def analysis(llm, transcript, params, question=None, template="", max_tokens=50):
 
    if question is None:
        prompt = PromptTemplate(template=template, input_variables=["transcript"])
        analysis_prompt = prompt.format(transcript=transcript)
    else:
        prompt = PromptTemplate(template=template, input_variables=["transcript", "question"])
        analysis_prompt = prompt.format(
            transcript=transcript,
            question=question
        )
    llm.model_kwargs = params

    print(colored(analysis_prompt, 'green'))
    response = llm(analysis_prompt)

    return response

def get_clues(res):
    
    def _extract_time(res):
        for item in res.split("\n"):
            if "시작시간:" in item:
                start_time = item.replace("시작시간:", "").strip()
            elif "종료시간:" in item:
                end_time = item.replace("종료시간:", "").strip()

        return start_time, end_time
    
    def _trim_audio_data(audio_file, save_file, start, end):
        sr = 96000
        y, sr = librosa.load(audio_file, sr=sr)
        ny = y[sr*(int(float(start))-3):sr*(int(float(end))+3)]
        sf.write(save_file, ny, 96000)
    
    start_time, end_time = _extract_time(res)
    
    print (start_time, end_time)
    _trim_audio_data("./records/voice-examples.wav", "./records/clue/clue.wav", start_time, end_time)

In [ ]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nHuman", "\n\n인간", "\n\n상담사", "\n\n\n", "\n\n질문"],
    "temperature":0,
    "top_p":0.999
}

### 3.3.1. Summary

In [ ]:
%%time

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    template=summary_template_ko
)

if not llm.streaming: print (res)

### 3.3.2. Question and Answer

In [ ]:
#%%time
question = "고객의 감정은 어떤가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

if not llm.streaming: print (res)

In [ ]:
%%time
question = "문제에 대한 개선을 위해서 어떤 방법이 있을까요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

if not llm.streaming: print (res)

In [ ]:
%%time
question = "학습지는 언제 종료되나요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_time_template_ko
)

if not llm.streaming: print (res)
get_clues(res)
ipd.Audio("./records/clue/clue.wav", autoplay=False)

In [ ]:
%%time
question = "환불은 언제 가능한가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_time_template_ko
)

if not llm.streaming: print (res)
get_clues(res)
ipd.Audio("./records/clue/clue.wav", autoplay=False)

In [ ]:
%%time
question = "결제된 금액은 얼마인가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_time_template_ko
)

if not llm.streaming: print (res)
get_clues(res)
ipd.Audio("./records/clue/clue.wav", autoplay=False)

In [ ]:
%%time
question = "상담원의 이름은 무엇인가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_time_template_ko
)

if not llm.streaming: print (res)
get_clues(res)
ipd.Audio("./records/clue/clue.wav", autoplay=False)

## 3.4. Handling long call transcripts
LLM의 인풋 토큰 한도를 초과하는 긴 문서를 처리하는 방법을 다룹니다. 

* Map Reduce
![nn](../../imgs/map_reduce.png)
출처: https://brain.d.foundation/Engineering/AI/Workaround+with+OpenAI's+token+limit+with+Langchain

* Refine
![nn](../../imgs/refine.png)
출처: https://brain.d.foundation/Engineering/AI/Workaround+with+OpenAI's+token+limit+with+Langchain

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

* prompting to divide and conquer

In [ ]:
stuff_prompt_template = """\n\nHuman:

다음 통화를 간단하게 요약해 주세요.

통화: {text}

요약:

\n\nAssistant:"""

chuck_prompt_template = """\n\nHuman:

다음 통화를 간단하게 요약해 주세요.

통화: {text}

요약:

\n\nAssistant:"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """\n\nHuman:

다음 통화를 간단하게 요약해 주세요.

통화: {text}

요약:

\n\nAssistant:"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

* summarize chain

In [ ]:
def summary_chain_init(chain_type, llm):


    combine_prompt = PromptTemplate(
        template=combine_prompt_template,
        input_variables=["text"]
    )

    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

In [ ]:
def long_call_analysis(llm, transcript, params, chain_type="MAP_REDUCE", max_tokens=50):

    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning

    print (num_tokens, max_tokens)

    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            #separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

        summary_chain = summary_chain_init(
            chain_type=chain_type, 
            llm=llm
        )
        response = summary_chain(
            {"input_documents": docs}
        )

        print ("Intermediate_steps: \n")
        for idx, step in enumerate(response["intermediate_steps"]):
            print (colored(f'step {idx}: \n', "green"))
            print (colored(f'{step}\n', "green"))

        return response["output_text"]

    else:

        prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])
        analysis_prompt = prompt.format(text=transcript)
        print (colored(analysis_prompt, 'green'))

        response = llm(analysis_prompt)

        return response


In [ ]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담사", "\n\n\n", "\n\n질문", "\n\nspk_0", "\n\n통화"],
    "temperature":0,
    "top_p":0.9
}

In [ ]:
%%time

res = long_call_analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    chain_type="REFINE" # REFINE, MAP_REDUCE
)


if not llm.streaming:
    print ("Results: \n")
    print (res)